# Modelling a simple RNA circuit

In this notebook, an example

In [19]:
from functools import partial
import numpy as np
from src.srv.io.manage.script_manager import script_preamble
from src.utils.common.setup import construct_circuit_from_cfg, prepare_config
from src.utils.circuit.agnostic_circuits.circuit_manager import CircuitModeller
from src.utils.modelling.physical import equilibrium_constant_reparameterisation


In [18]:
np.random.randint(0, 5, (3,3))

array([[1, 2, 0],
       [2, 3, 2],
       [2, 3, 4]])

In [11]:
fake_interactions = np.random.randint(-50, 0, (3,3))
config = {}
config = {
    "data_path": "data/example/toy_mRNA_circuit.fasta",
    "experiment": {
        "purpose": "example"
    },
    "interaction_simulator": {
        "name": "IntaRNA",
        "postprocess": True
    },
    "interactions_loaded": True,
    "interactions": {
        "energies": fake_interactions,
        "eqconstants": equilibrium_constant_reparameterisation(fake_interactions, initial=100),
        "binding_rates_dissociation": np.ones((3,3)),
        },
    "molecular_params": "src/utils/common/configs/RNA_circuit/molecular_params.json",
    "simulation": {
        "dt": 0.01,
        "t0": 0,
        "t1": 100,
        "solver": "jax"
    },
    "signal": {
        "inputs": ["RNA_0"],
        "outputs": ["RNA_1"],
        "function_name": "step_function",
        "function_kwargs": {
            "impulse_center": 40, 
            "impulse_halfwidth": 5, 
            "target": 0.5
        } 
    },
    "system_type": "RNA"
}

config, data_writer = script_preamble(config=config, data_writer=None, alt_cfg_filepath=None)
config = prepare_config(config)

In [14]:
circuit = construct_circuit_from_cfg(config_file=config, prev_configs=None)
circuit.__dict__

{'name': 'toy_mRNA_circuit',
 'subname': 'ref_circuit',
 'result_collector': <src.utils.results.results.ResultCollector at 0x7f9532bbefe0>,
 'use_prod_and_deg': True,
 'model': MedModel(species=[Species: ('RNA_0', 'RNA_0'), Species: ('RNA_0', 'RNA_1'), Species: ('RNA_0', 'RNA_2'), Species: ('RNA_1', 'RNA_1'), Species: ('RNA_1', 'RNA_2'), Species: ('RNA_2', 'RNA_2'), Species: RNA_0, Species: RNA_1, Species: RNA_2], reactions=[Reaction(input=[Species: RNA_0], output=[], forward_rate=0.01175, reverse_rate=0), Reaction(input=[Species: RNA_1], output=[], forward_rate=0.01175, reverse_rate=0), Reaction(input=[Species: RNA_2], output=[], forward_rate=0.01175, reverse_rate=0), Reaction(input=[Species: ('RNA_0', 'RNA_0')], output=[], forward_rate=0.01175, reverse_rate=0), Reaction(input=[Species: ('RNA_0', 'RNA_1')], output=[], forward_rate=0.01175, reverse_rate=0), Reaction(input=[Species: ('RNA_0', 'RNA_2')], output=[], forward_rate=0.01175, reverse_rate=0), Reaction(input=[Species: ('RNA_1',

In [ ]:

modeller = CircuitModeller(result_writer=data_writer, config=config)
circuit = modeller.init_circuit(circuit=circuit)
circuit = modeller.simulate_signal(circuit=circuit, signal=circuit.signal, solver=config['simulation']['solver'])
modeller.write_results(circuit)


In [15]:
circuit.result_collector.results

{}